In [21]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin")

<AddedDllDirectory('C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin')>

In [22]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [23]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [24]:
path = r"..\data\KAGGLE\kaggle_finalised.csv"
df = pd.read_csv(path, usecols=["source","labels","headline","text"])
pd.set_option('display.max_colwidth', -1)
x_train, x_test = train_test_split(df, test_size=0.2, random_state=111)

C:\Users\Sam's PC\AppData\Local\Temp\ipykernel_17336\3817716175.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [25]:
print(df['labels'].value_counts())
swingdata = df['labels']

 0    5764
 1    1297
-1    1120
 2    633 
-2    529 
 3    158 
-3    119 
 4    63  
-4    30  
Name: labels, dtype: int64


In [26]:
class_weights = list(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(df['labels']), y=swingdata))
np.unique(df['labels'])
class_weights.sort()
df['labels'].value_counts()

 0    5764
 1    1297
-1    1120
 2    633 
-2    529 
 3    158 
-3    119 
 4    63  
-4    30  
Name: labels, dtype: int64

In [27]:
weights={}

for index, weight in enumerate(class_weights):
    weights[index]=weight

In [28]:
dataset_train = tf.data.Dataset.from_tensor_slices((x_train['text'].values, x_train['labels'].values))
dataset_test = tf.data.Dataset.from_tensor_slices((x_test['text'].values, x_test['labels'].values))

In [29]:
def fetch(text, labels):
    return text, tf.one_hot(labels, 9)

In [30]:
train_data_f=dataset_train.map(fetch)
test_data_f=dataset_test.map(fetch)

In [31]:
train_data, train_labels = next(iter(train_data_f.batch(5)))

In [32]:
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, output_shape=[128], input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 1.5701346e+00,  5.7551455e-01, -9.1829829e-02, -2.9152185e-01,
        -2.1295413e-01, -2.9568093e-02,  1.7536528e-02, -2.3557460e-01,
        -6.8924934e-02,  8.6796895e-02,  4.0287706e-01,  4.0075305e-01,
        -1.5786774e-01,  2.0713823e-01, -2.3699395e-01,  1.4814676e-01,
        -4.6862227e-01, -2.1087460e-01, -9.7300753e-02,  1.7832458e+00,
         1.2497991e-01,  4.1937110e-01, -1.2303090e-01,  9.2001259e-02,
        -9.6788570e-02,  1.7181440e-01,  3.9333358e-01, -5.9749130e-02,
        -8.4132716e-02, -9.5848618e-03, -2.4290852e-01, -1.7437834e-01,
         2.0996188e-01, -2.3239635e-02,  1.5073279e-01,  1.9917414e-01,
         8.2096960e-03, -2.6212114e-01, -1.7529175e-01, -1.8709232e-01,
         1.2903674e-01, -3.4300316e-02, -1.8365154e-01,  3.5388574e-01,
         1.9681220e-01, -1.3587591e-01, -8.1963144e-02, -1.6614601e-01,
        -8.1249990e-02, -2.7942288e-01, -9.5973194e-02,  9.2167236e-02,
         3.027

In [33]:
model = tf.keras.Sequential()
#model.add(hub_layer)
#for units in [128, 128, 64, 32]:
#    model.add(tf.keras.layers.Dense(units, activation='relu'))
#    model.add(tf.keras.layers.Dropout(0.4))
#model.add(tf.keras.layers.Dense(9, activation="softmax"))
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(1000,activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None))
model.add(tf.keras.layers.Dense(500,activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None))
model.add(tf.keras.layers.Dense(120,activation=tf.nn.sigmoid))
model.add(tf.keras.layers.Dropout(0.5, noise_shape=None, seed=None))
model.add(tf.keras.layers.Dense(9,activation=tf.nn.softmax))


model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 128)               124642688 
                                                                 
 dense_4 (Dense)             (None, 1000)              129000    
                                                                 
 dropout_3 (Dropout)         (None, 1000)              0         
                                                                 
 dense_5 (Dense)             (None, 500)               500500    
                                                                 
 dropout_4 (Dropout)         (None, 500)               0         
                                                                 
 dense_6 (Dense)             (None, 120)               60120     
                                                                 
 dropout_5 (Dropout)         (None, 120)              

In [34]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [35]:
train_data_f=train_data_f.shuffle(70000).batch(512)
test_data_f=test_data_f.batch(512)

In [36]:
history = model.fit(
    train_data_f,
    epochs=15,
    validation_data=test_data_f,
    verbose=1,
    class_weight=weights
)

Epoch 1/15


C:\Users\Sam's PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5531: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


16/16 [==============================] - 3s 145ms/step - loss: 0.5579 - accuracy: 0.3664 - val_loss: 1.0201 - val_accuracy: 0.1318
Epoch 2/15
16/16 [==============================] - 2s 116ms/step - loss: 0.5088 - accuracy: 0.3822 - val_loss: 1.0110 - val_accuracy: 0.7849
Epoch 3/15
16/16 [==============================] - 2s 124ms/step - loss: 0.5002 - accuracy: 0.4075 - val_loss: 1.0070 - val_accuracy: 0.1462
Epoch 4/15
16/16 [==============================] - 2s 123ms/step - loss: 0.4987 - accuracy: 0.3945 - val_loss: 0.9744 - val_accuracy: 0.7854
Epoch 5/15
16/16 [==============================] - 2s 118ms/step - loss: 0.4895 - accuracy: 0.4923 - val_loss: 1.0242 - val_accuracy: 0.1318
Epoch 6/15
16/16 [==============================] - 2s 121ms/step - loss: 0.4897 - accuracy: 0.3997 - val_loss: 0.9411 - val_accuracy: 0.7849
Epoch 7/15
16/16 [==============================] - 2s 124ms/step - loss: 0.4829 - accuracy: 0.4812 - val_loss: 0.9645 - val_accuracy: 0.5610
Epoch 8/15
16/16 

In [39]:
loss, accuracy = model.evaluate(test_data_f)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

4/4 [==============================] - 0s 55ms/step - loss: 0.7522 - accuracy: 0.6032
Loss:  0.7521812915802002
Accuracy:  0.6031909584999084
